In [5]:
import os
import pandas as pd

# Diretório com os arquivos
path = "results_models/results_5_years/"
files = [f for f in os.listdir(path) if f.startswith("plot_results_")]

# Dicionário para mapear nomes de modelos bonitos e GRANU
model_mapping = {
    "lstm": ("LSTM", "LSTM_FT_Indiv"),
    "nbeats": ("N-BEATS", "NBEATS_FT_Indiv"),
    "times_fm": ("TimesFM", "TimesFM_ZS"),
    "morai_moe": ("Morai-MoE", "Morai-MoE_ZS"),
    "timemoe": "TimeMoE"
}

timemoe_granu_mapping = {
    "TimeMoE-50M_ZERO_SHOT_5_YEARS": "TimeMoE_50M_ZS",
    "TimeMoE-200M_ZERO_SHOT_5_YEARS": "TimeMoE_200M_ZS",
    "TimeMoE-50M_FINE_TUNING_INDIV_5_YEARS": "TimeMoE_50M_FT_Indiv",
    "TimeMoE-200M_FINE_TUNING_INDIV_5_YEARS": "TimeMoE_200M_FT_Indiv",
    "TimeMoE-50M_FINE_TUNING_GLOBAL_5_YEARS": "TimeMoE_50M_FT_Global",
    "TimeMoE-200M_FINE_TUNING_GLOBAL_5_YEARS": "TimeMoE_200M_FT_Global",
    "TimeMoE-50M-FINE-TUNING-PRODUCT": "TimeMoE_50M_FT_Product",
    "TimeMoE-200M-FINE-TUNING-PRODUCT": "TimeMoE_200M_FT_Product"
}

all_data = []

for file in files:
    df = pd.read_csv(os.path.join(path, file))
    
    # Identificar o modelo pelo nome do arquivo
    model_key = file.replace("plot_results_", "").replace(".csv", "").lower()
    
    if "timemoe" in model_key:
        # Para TimeMoE, identificamos pelo TYPE_PREDICTIONS
        df["GRANU"] = df["TYPE_PREDICTIONS"].map(timemoe_granu_mapping)
        df["MODEL"] = "TimeMoE"
    else:
        model, granu = model_mapping.get(model_key, (None, None))
        df["MODEL"] = model
        df["GRANU"] = granu
    
    df.rename(columns={
        "STATE": "UF",
        "PRODUCT": "PRODUCT",
        "TYPE_PREDICTIONS": "PARAMETERS",
        "MAPE": "MAPE",
        "POCID": "POCID",
        "PBE": "PBE",
        "MASE(2020,2021,2022,2023,2024)": "MASE"
    }, inplace=True)
    
    df["CLASS"] = "2 - DL"
    
    # Separar os pontos
    pontos = df["PONTOS"].str.split(",", expand=True)
    pontos.columns = [f"P{i+1}" for i in range(60)]
    
    # Separar MASE em 5 colunas
    mase_cols = df["MASE"].str.split(",", expand=True)
    mase_cols.columns = ["MASE1", "MASE2", "MASE3", "MASE4", "MASE5"]
    
    df = df.drop(columns=["PONTOS", "MASE"]).join(mase_cols).join(pontos)
    all_data.append(df)

# Concatenar todos os dados
final_df = pd.concat(all_data, ignore_index=True)

column_order = [
    "PRODUCT", "UF", "MODEL", "PARAMETERS", "GRANU", "CLASS", "MAPE", "POCID", "PBE",
    "MASE1", "MASE2", "MASE3", "MASE4", "MASE5", "RRMSE"
] + [f"P{i}" for i in range(1, 61)]

# Reordenar as colunas do DataFrame
final_df = final_df[column_order]

# Salvar o resultado final
final_df.to_csv("results_models/merged_results.csv", index=False)
